In [1]:
import tensorflow as tf
from tensorflow import keras
import gym

In [2]:
env = gym.make("CartPole-v1")
obs = env.reset(seed=42)

In [3]:
obs

array([ 0.0273956 , -0.00611216,  0.03585979,  0.0197368 ], dtype=float32)

In [4]:
try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다